# Keras Model Save / Load 

1. Usually we just train until it starts to overfit rather than fixed epoch
2. and we can save model to use later
3. Only works for functional and sequential API

```
model = new model
model.compile(..)
model.fit(..)
model.save("model.h5")
```

## Prepare Model

In [3]:
import tensorflow
from tensorflow import keras ## Use Keras from Tensorflow to work
import matplotlib.pyplot as plt

In [4]:
# Load Dataset (fashion MNIST)

fashion = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion.load_data()

X_train_full.shape, y_train_full.shape

((60000, 28, 28), (60000,))

In [5]:
X_train_scaled = X_train_full.astype('float32')/255.0

In [6]:
X_valid, X_train = X_train_scaled[:5000], X_train_scaled[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

## Checkpoints

In [5]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]), # So that we don't have to do .reshape(-1,1)
    keras.layers.Dense(100, activation="relu"),  # Hidden (1)
    keras.layers.Dense(100, activation="relu"),  # Hidden (2)
    keras.layers.Dense(100, activation="relu"),  # Hidden (3)
    keras.layers.Dense(100, activation="relu"),  # Hidden (4)
    keras.layers.Dense(10, activation="softmax") # Output Layer, 10 classes, decide with softmax
                                                 # Use sigmoid for 2
])
model.compile(loss="sparse_categorical_crossentropy", 
              optimizer="sgd",
              metrics=["accuracy"])

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
## This will overwrite and save only best weight but will not stop
checkpoint_cb = keras.callbacks.ModelCheckpoint("out/fashion_cp.h5",
                                                save_best_only=True)


history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb])

Train on 55000 samples, validate on 5000 samples
Epoch 1/20
55000/55000 [==============================] - 8s 139us/sample - loss: 0.8252 - acc: 0.7136 - val_loss: 0.5209 - val_acc: 0.8228
Epoch 2/20
55000/55000 [==============================] - 7s 129us/sample - loss: 0.5085 - acc: 0.8207 - val_loss: 0.4590 - val_acc: 0.8390
Epoch 3/20
55000/55000 [==============================] - 7s 134us/sample - loss: 0.4498 - acc: 0.8403 - val_loss: 0.4110 - val_acc: 0.8566
Epoch 4/20
55000/55000 [==============================] - 7s 133us/sample - loss: 0.4165 - acc: 0.8535 - val_loss: 0.3845 - val_acc: 0.8630
Epoch 5/20
55000/55000 [==============================] - 8s 144us/sample - loss: 0.3920 - acc: 0.8596 - val_loss: 0.3789 - val_acc: 0.8664
Epoch 6/20
55000/55000 [==============================] - 8s 148us/sample - loss: 0.3744 - acc: 0.8674 - val_loss: 0.3820 - val_acc: 0.8684
Epoch 7/20
55000/55000 [==============================] - 8s 153us/sample - loss: 0.3623 - acc: 0.8690 - val_lo

In [10]:
## Callback: Tensorboard
import os
root_logdir = os.path.join(os.curdir, "out/tensorboard/fashion")

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]), # So that we don't have to do .reshape(-1,1)
    keras.layers.Dense(100, activation="relu"),  # Hidden (1)
    keras.layers.Dense(100, activation="relu"),  # Hidden (2)
    keras.layers.Dense(100, activation="relu"),  # Hidden (3)
    keras.layers.Dense(100, activation="relu"),  # Hidden (4)
    keras.layers.Dense(10, activation="softmax") # Output Layer, 10 classes, decide with softmax
                                                 # Use sigmoid for 2
])

tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [11]:
model.compile(loss="sparse_categorical_crossentropy", 
              optimizer="sgd",
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid),
                    callbacks=[tensorboard_cb])

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 8s 140us/sample - loss: 0.8449 - acc: 0.7003 - val_loss: 0.5562 - val_acc: 0.7996
Epoch 2/10
55000/55000 [==============================] - 7s 133us/sample - loss: 0.4954 - acc: 0.8255 - val_loss: 0.4467 - val_acc: 0.8448
Epoch 3/10
55000/55000 [==============================] - 7s 130us/sample - loss: 0.4415 - acc: 0.8436 - val_loss: 0.4297 - val_acc: 0.8474
Epoch 4/10
55000/55000 [==============================] - 7s 133us/sample - loss: 0.4093 - acc: 0.8532 - val_loss: 0.4168 - val_acc: 0.8528
Epoch 5/10
55000/55000 [==============================] - 8s 140us/sample - loss: 0.3858 - acc: 0.8611 - val_loss: 0.3653 - val_acc: 0.8668
Epoch 6/10
55000/55000 [==============================] - 8s 138us/sample - loss: 0.3692 - acc: 0.8664 - val_loss: 0.3549 - val_acc: 0.8714
Epoch 7/10
55000/55000 [==============================] - 8s 141us/sample - loss: 0.3561 - acc: 0.8697 - val_lo

### Run tensorboard

Tensorboard will detect subdirectory automatically
```
$ tensorboard --log-dir=out/tensorboard/fashion
```

## Early stopping 

In [13]:
## Stop the training when there's no more progress, and restore best weight
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                                  restore_best_weights=True)

## We actualy don't need to do this as early stopping will restore weights at the end
checkpoint_cb = keras.callbacks.ModelCheckpoint("out/fashion_cp.h5",
                                                save_best_only=True)


run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]), # So that we don't have to do .reshape(-1,1)
    keras.layers.Dense(300, activation="relu"),  # Hidden (1)
    keras.layers.Dense(300, activation="relu"),  # Hidden (2)
    keras.layers.Dense(300, activation="relu"),  # Hidden (3)
    keras.layers.Dense(300, activation="relu"),  # Hidden (4)
    keras.layers.Dense(10, activation="softmax") # Output Layer, 10 classes, decide with softmax
                                                 # Use sigmoid for 2
])
model.compile(loss="sparse_categorical_crossentropy", 
              optimizer="sgd",
              metrics=["accuracy"])

In [14]:
history = model.fit(X_train, y_train, epochs=1000,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb, early_stopping_cb, tensorboard_cb])

Train on 55000 samples, validate on 5000 samples
Epoch 1/1000
55000/55000 [==============================] - 9s 171us/sample - loss: 0.7609 - acc: 0.7415 - val_loss: 0.5577 - val_acc: 0.7924
Epoch 2/1000
55000/55000 [==============================] - 9s 170us/sample - loss: 0.4797 - acc: 0.8301 - val_loss: 0.4568 - val_acc: 0.8408
Epoch 3/1000
55000/55000 [==============================] - 10s 178us/sample - loss: 0.4239 - acc: 0.8497 - val_loss: 0.3924 - val_acc: 0.8684
Epoch 4/1000
55000/55000 [==============================] - 10s 188us/sample - loss: 0.3907 - acc: 0.8606 - val_loss: 0.3656 - val_acc: 0.8712
Epoch 5/1000
55000/55000 [==============================] - 10s 175us/sample - loss: 0.3664 - acc: 0.8686 - val_loss: 0.3567 - val_acc: 0.8730
Epoch 6/1000
55000/55000 [==============================] - 9s 172us/sample - loss: 0.3480 - acc: 0.8735 - val_loss: 0.3486 - val_acc: 0.8758
Epoch 7/1000
55000/55000 [==============================] - 10s 184us/sample - loss: 0.3311 - ac

In [15]:
model.save("out/fashion_earlystop.h5")